In [1]:
from dash import Dash, dcc, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import re
from functools import reduce
app = Dash(__name__)

In [2]:
# Top world's companies
df_twc = pd.read_csv('..\Data\Top_Worlds_Companies\companies.csv') # updated 2 months ago; source: kaggle <- ambitionbox
def contains_numeric(string):
    pattern = r'\d'
    match = re.search(pattern, string)
    if match:
        return np.nan
    elif string == 'Ahmedabad':
        return np.nan
    else:
        return string
def string_to_num(amount):
    r = 0
    try:
        if amount[-1]=='k':
            r = 1000*float(amount[:-1])
        else:
            r = float(amount)
        return r
    except ValueError:
        return np.nan
    
df_twc['desc'] = df_twc['Description'].str.split('|').str[0]
df_twc['sector'] = df_twc['desc'].apply(contains_numeric)
df_twc.drop('desc',axis=1, inplace=True)
df_twc['HRF'] = df_twc['Highly_rated_for'].str.split(', ')
df_twc['Avg_salary'] = df_twc['Avg_salary'].apply(string_to_num)
df_twc['Total_jobs_available'] = df_twc['Total_jobs_available'].apply(string_to_num)
df_twc['Interviews_taken'] = df_twc['Interviews_taken'].apply(string_to_num)
hrf_ser = df_twc['HRF'].dropna()
union_HRF = reduce(set.union, map(set, hrf_ser))


In [3]:
# styles
colors = {
    'background': '#d9ffb3',
    'text': '#0066cc',
    'title' : '#000066'
}

In [4]:
# companies that you might like to work in based off sectors, ratings and positive critiques

@app.callback(
    Output("fig1", "figure"),
    Input("sector-filter", "value"),
    Input("critique-filter", "value")
)
def update_Sect_Ratings(sector, crit):
    # Apply filters to the data
    filtered_df = df_twc
    filtered_df.dropna(subset=['sector','HRF'], inplace=True)
    if sector:
        filtered_df = filtered_df[filtered_df["sector"].isin(sector)]
    if crit:
        filtered_df = filtered_df[filtered_df["HRF"].apply(lambda x: bool(set(x) & set(crit)))]

    # Create the bar chart
    full_fig = make_subplots(rows=2, cols=1,)
    fig1 = px.bar(filtered_df.sort_values("Avg_salary", ascending=False).head(20),
                 x="Company_name",
                 y="Avg_salary",
                 color="Ratings",
                 hover_data=["Avg_salary"],
                 )
    fig1.update_xaxes(showgrid=False)
    fig1.update_yaxes(showgrid=False)
    full_fig.update_yaxes(title_text="Average salary", row=1, col=1)
    full_fig.update_layout(
    coloraxis_colorbar=dict(len=0.4, x=1.1, y=0.8), coloraxis_colorscale='ice'
    )
    fig2 = px.scatter(filtered_df.sort_values("Avg_salary", ascending=False).head(20), x='Total_jobs_available', y='Interviews_taken', 
                      size='Ratings',hover_data=["Company_name"])
    full_fig.update_xaxes(title_text="Number of jobs", row=2, col=1)
    full_fig.update_yaxes(title_text="Number of interviews", row=2, col=1)
    full_fig.add_trace(fig1.data[0], row=1,col=1)
    full_fig.add_trace(fig2.data[0], row=2, col=1)
    full_fig.update_layout(
    width=1200,   # Overall width of the subplot
    height=700,   # Overall height of the subplot
    margin=dict(l=20, r=20, t=40, b=20),  # Adjust margins
    font=dict(color=colors['text']),
    plot_bgcolor=colors['background'], paper_bgcolor=colors['background']
    )
    return full_fig

In [ ]:
app.layout = html.Div(style={'backgroundColor': colors['background'], 'margin': 10, 'padding': 20}, children=[
    html.H1(
        children='Companies based off your likings',
        style={
            'textAlign': 'center',
            'color': colors['title']
        }
    ),

     dbc.Row([
                html.Label("Sector", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="sector-filter",
                    options=[{"label": s, "value": s}
                             for s in df_twc['sector'].dropna().unique()],
                    value=None,
                    multi=True,
                    style={
                        "width": "100%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
                html.Label("Features", style={'color': colors['text']}),
                dcc.Dropdown(
                    id="critique-filter",
                    options=[{"label": l, "value": l}
                             for l in union_HRF],
                    value=None,
                    multi=True,
                    style={
                        "width": "100%",  # set the width of the dropdown to 50% of its container
                        "height": "50%",  # set the height of each dropdown item to 40 pixels
                        "backgroundColor": colors['background'],  # set the background color of the dropdown to dark gray
                        "color": "#000",  # set the font color of the dropdown to light gray
                        "fontFamily": "sans-serif"  # set the font family of the dropdown to sans-serif
                    }
                ),
         dcc.Graph(id="fig1")
     ])
])

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)